# Chess RL Agent - Training (Optimized)

**New:** Shaped rewards + speed optimizations (~60% faster, more reliable convergence)

**Strategy:** Split into safe 3-4 hour sessions (no browser babysitting needed)

**Workflow:**
1. Run cells 1-6 once → Auto-trains or resumes (~3-4 hours per session)
2. Training completes, checkpoints auto-backup to Drive
3. Close browser, take a break
4. Next session: Re-run cells 1-6 → Automatically resumes where you left off

**Total time:** 2-3 sessions × 3-4 hours = 8-12 hours for 10 iterations

**What's optimized:**
- Shaped rewards (material + pawn advancement + activity) prevent draw loops
- Smaller network (48 channels, 3 blocks) for faster inference
- Asymmetric MCTS (12 sims self-play, 40 arena) saves time without sacrificing quality
- Adaptive schedule (progressive games/epochs/lr) optimizes learning
- Early stopping skips wasted epochs

## 1. Verify GPU

In [ ]:
import torch

print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"CUDA: {torch.cuda.is_available()}")

if not torch.cuda.is_available():
    print("\n⚠ Go to Runtime → Change runtime type → Select GPU (L4 recommended)")
else:
    print("\n✓ GPU ready")

## 2. Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/chess_checkpoints

print("✓ Drive mounted")

## 3. Clone Repository

In [ ]:
!rm -rf rl_chess_agent
!git clone https://github.com/capacap/rl_chess_agent.git
%cd rl_chess_agent

print("✓ Repository cloned")

## 4. Install Dependencies

In [ ]:
# Install chess library (Colab has torch, numpy, etc.)
!pip install -q -r requirements-colab.txt

# Verify imports
import chess
from model.network import ChessNet

print(f"✓ Dependencies installed")
print(f"  chess: {chess.__version__}")
print(f"  torch: {torch.__version__}")

## 5. Configure Training

In [ ]:
# === Project Configuration ===
# Set this once and keep it the same across all sessions
PROJECT_NAME = "proof_of_concept"  # Change this for different training runs

# === Session Configuration ===
SESSION_ITERATIONS = 5    # Iterations per session (3-4 hours each with optimizations)
TOTAL_TARGET = 10         # Total iterations you want overall

# === Optimized Training Parameters ===
# These defaults use shaped rewards + speed optimizations (60% faster!)

# Network (smaller = faster)
CHANNELS = 48             # Down from 64 (~500K params, fast inference)
BLOCKS = 3                # Down from 4

# MCTS (asymmetric: fewer sims in self-play, more in arena)
SIMULATIONS = 12          # Self-play sims (down from 20)
SIMULATIONS_ARENA = 40    # Arena sims (keep high for evaluation)

# Schedule (adaptive by default, progressively more games/epochs)
USE_ADAPTIVE = True       # Use progressive schedule from config.py
ARENA_GAMES = 16          # Down from 20 (adjusted threshold for statistical significance)

# Advanced (only change if disabling adaptive schedule)
BATCH_SIZE = 256          # Ignored if adaptive schedule enabled
EPOCHS = 5                # Ignored if adaptive schedule enabled  
LEARNING_RATE = 1e-3      # Ignored if adaptive schedule enabled

# Checkpoint directories
CHECKPOINT_DIR = f"checkpoints/{PROJECT_NAME}"
GDRIVE_BACKUP = "/content/drive/MyDrive/chess_checkpoints"

print("Training Configuration (Optimized):")
print(f"  Project: {PROJECT_NAME}")
print(f"  Target: {TOTAL_TARGET} iterations total")
print(f"  Session size: {SESSION_ITERATIONS} iterations (~{SESSION_ITERATIONS * 0.8:.0f}-{SESSION_ITERATIONS * 1.3:.0f} hours)")
print(f"\nOptimizations enabled:")
print(f"  - Shaped rewards (prevents draw loops)")
print(f"  - Smaller network ({CHANNELS} channels, {BLOCKS} blocks)")
print(f"  - Asymmetric MCTS ({SIMULATIONS} sims self-play, {SIMULATIONS_ARENA} arena)")
print(f"  - Adaptive schedule (progressive games/epochs/lr)")
print(f"  - Early stopping (skip wasted epochs)")
print(f"\nExpected: ~60% faster than baseline, more reliable convergence")
print(f"\nCheckpoints: {CHECKPOINT_DIR}")
print(f"Drive backup: {GDRIVE_BACKUP}/{PROJECT_NAME}/")

## 6. Train (auto-resumes if checkpoint exists)

In [ ]:
import os
import glob

# Check for existing checkpoints
existing_checkpoints = sorted(glob.glob(f"{CHECKPOINT_DIR}/iteration_*.pt"))

if existing_checkpoints:
    # Resume from latest checkpoint
    latest = existing_checkpoints[-1]
    completed = int(latest.split('_')[-1].split('.')[0])
    remaining = TOTAL_TARGET - completed
    
    print("="*60)
    print("RESUMING TRAINING (Optimized)")
    print("="*60)
    print(f"  Progress: {completed}/{TOTAL_TARGET} iterations complete")
    print(f"  Resuming from: {os.path.basename(latest)}")
    
    if remaining > 0:
        iterations_this_session = min(SESSION_ITERATIONS, remaining)
        print(f"  This session: {iterations_this_session} iterations (~{iterations_this_session * 0.8:.0f}-{iterations_this_session * 1.3:.0f} hours)")
        print(f"  Remaining after: {remaining - iterations_this_session} iterations\n")
        
        # Resume training with optimized parameters
        if USE_ADAPTIVE:
            !python train.py \
              --resume {latest} \
              --iterations {iterations_this_session} \
              --simulations {SIMULATIONS} \
              --simulations-arena {SIMULATIONS_ARENA} \
              --channels {CHANNELS} \
              --blocks {BLOCKS} \
              --checkpoint-dir {CHECKPOINT_DIR} \
              --gdrive-backup-dir {GDRIVE_BACKUP}
        else:
            !python train.py \
              --resume {latest} \
              --iterations {iterations_this_session} \
              --simulations {SIMULATIONS} \
              --simulations-arena {SIMULATIONS_ARENA} \
              --channels {CHANNELS} \
              --blocks {BLOCKS} \
              --games-per-iter {GAMES_PER_ITER} \
              --arena-games {ARENA_GAMES} \
              --batch-size {BATCH_SIZE} \
              --epochs {EPOCHS} \
              --lr {LEARNING_RATE} \
              --no-adaptive-schedule \
              --checkpoint-dir {CHECKPOINT_DIR} \
              --gdrive-backup-dir {GDRIVE_BACKUP}
        
        new_completed = completed + iterations_this_session
        print(f"\n{'='*60}")
        print(f"SESSION COMPLETE")
        print(f"{'='*60}")
        print(f"  Progress: {new_completed}/{TOTAL_TARGET} iterations")
        print(f"  Checkpoints: {GDRIVE_BACKUP}/{PROJECT_NAME}/")
        
        if new_completed >= TOTAL_TARGET:
            print(f"\n🎉 TRAINING COMPLETE!")
            print(f"  Final model: {CHECKPOINT_DIR}/iteration_{TOTAL_TARGET}.pkl")
        else:
            print(f"\nNext steps:")
            print(f"  1. Close browser (checkpoints saved to Drive)")
            print(f"  2. When ready: Re-run cells 1-6 to continue")
            print(f"  3. Remaining: {TOTAL_TARGET - new_completed} iterations")
    else:
        print(f"\n🎉 TRAINING ALREADY COMPLETE!")
        print(f"  Final model: {CHECKPOINT_DIR}/iteration_{TOTAL_TARGET}.pkl")
        print(f"  Download from: {GDRIVE_BACKUP}/{PROJECT_NAME}/")

else:
    # Start fresh training
    print("="*60)
    print("STARTING NEW TRAINING (Optimized)")
    print("="*60)
    print(f"  Project: {PROJECT_NAME}")
    print(f"  This session: {SESSION_ITERATIONS} iterations (~{SESSION_ITERATIONS * 0.8:.0f}-{SESSION_ITERATIONS * 1.3:.0f} hours)")
    print(f"  Total target: {TOTAL_TARGET} iterations\n")
    
    # Start training with optimized parameters
    if USE_ADAPTIVE:
        !python train.py \
          --iterations {SESSION_ITERATIONS} \
          --simulations {SIMULATIONS} \
          --simulations-arena {SIMULATIONS_ARENA} \
          --channels {CHANNELS} \
          --blocks {BLOCKS} \
          --checkpoint-dir {CHECKPOINT_DIR} \
          --gdrive-backup-dir {GDRIVE_BACKUP}
    else:
        !python train.py \
          --iterations {SESSION_ITERATIONS} \
          --simulations {SIMULATIONS} \
          --simulations-arena {SIMULATIONS_ARENA} \
          --channels {CHANNELS} \
          --blocks {BLOCKS} \
          --games-per-iter {GAMES_PER_ITER} \
          --arena-games {ARENA_GAMES} \
          --batch-size {BATCH_SIZE} \
          --epochs {EPOCHS} \
          --lr {LEARNING_RATE} \
          --no-adaptive-schedule \
          --checkpoint-dir {CHECKPOINT_DIR} \
          --gdrive-backup-dir {GDRIVE_BACKUP}
    
    print(f"\n{'='*60}")
    print(f"SESSION COMPLETE")
    print(f"{'='*60}")
    print(f"  Progress: {SESSION_ITERATIONS}/{TOTAL_TARGET} iterations")
    print(f"  Checkpoints: {GDRIVE_BACKUP}/{PROJECT_NAME}/")
    
    if SESSION_ITERATIONS >= TOTAL_TARGET:
        print(f"\n🎉 TRAINING COMPLETE!")
        print(f"  Final model: {CHECKPOINT_DIR}/iteration_{TOTAL_TARGET}.pkl")
    else:
        print(f"\nNext steps:")
        print(f"  1. Close browser (checkpoints saved to Drive)")
        print(f"  2. When ready: Re-run cells 1-6 to continue")
        print(f"  3. Remaining: {TOTAL_TARGET - SESSION_ITERATIONS} iterations")

## Troubleshooting

**Out of memory:** 
- Reduce `CHANNELS = 32` or `BLOCKS = 2` (smaller network)
- Or disable adaptive schedule and set `BATCH_SIZE = 128`

**Too slow:** 
- Reduce `SIMULATIONS = 10` (self-play) or `SIMULATIONS_ARENA = 30` (arena)
- Or set `USE_ADAPTIVE = False` and reduce `GAMES_PER_ITER`

**Disable optimizations:**
- Set `USE_ADAPTIVE = False` to use fixed parameters
- Set `CHANNELS = 64, BLOCKS = 4` for original network size
- Set `SIMULATIONS = SIMULATIONS_ARENA = 20` for symmetric MCTS

**Start over:** Change `PROJECT_NAME` in cell 5 to create a new training run

**Download checkpoints:** Already in Google Drive at `/MyDrive/chess_checkpoints/{PROJECT_NAME}/`

**Check progress:** Look at `training.log` in checkpoints for detailed metrics